In [2]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
import re
import random

# import yelp client library
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator


RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

ImportError: numpy.core.multiarray failed to import

In [3]:
def authenticate(config_filepath):
    """
    Create an authenticated yelp-python client.

    Args:
        config_filepath (string): relative path (from this file) to a file with your Yelp credentials

    Returns:
        client (yelp.client.Client): authenticated instance of a yelp.Client
    """
    
    creds = json.load(open(config_filepath))
    auth = Oauth1Authenticator(**creds)
    return Client(auth)

In [4]:
def yelp_search(client, query):
    """
    Make an authenticated request to the Yelp API.

    Args:
        query (string): Search term

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the query
        businesses (list): list of yelp.obj.business.Business objects
    """
    
    # Write solution here
    res = client.search(query)
    return (res.total, res.businesses)

In [5]:
def all_restaurants(client, query, food):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of yelp.obj.business.Business objects
    """
    B = []
    while 1:
        try:
            res = client.search(query,**{"category_filter":food,"offset" : len(B)})
            if not res.businesses:
                return B
            B += res.businesses
#            time.sleep(.1)
        except:
            return B

In [6]:
def get_cities_info(client, food, cities):
    d = {}
    for city in cities:
        print "requesting: ", city,
        d["%s_%s"%(city,food)] = all_restaurants(client,city,food)
        print ": done"
    #d = {("%s_%s"%(city,food)):all_restaurants(client,city,food) for city in cities}
    valids = filter(lambda x: len(d[x]) >= 10, d)
    res = {v:d[v] for v in valids}
    return res

def get_all_cities():
    with open("cities.txt",'r') as f:
        lines = map(lambda l: ", ".join(l.rstrip().split("\t")[1:3]),f.readlines())
        return lines
    
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
def save_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
        
def get_city_data(data, city, foodtype):
    '''returns dictionary:
        avg_review
        num_restaurants
        city
    '''
    result = {}
    key = "%s_%s"%(city,foodtype)
    info = data[key]
    result["num_restaurants"] = len(info)
    result["avg_review"] = sum(map(lambda x: x.rating, info))/float(len(info))
    return result
    
def get_all_city_data(data):
    result = {}
    for city_food in data:
        city = city_food.split("_")[0]
        foodtype = city_food.split("_")[1]
        result[city] = get_city_data(data,city,foodtype)
    return result

def clean_business(b):
    valids = ["categories", "location", "name", "rating", "review_count","url"]
    return {x : b.__dict__[x] for x in valids}
    
    
def clean_data(data):
    new_data = {}
    for city in data:
        new_data[city] = map(clean_business, data[city])
    return new_data

def add_feature_info(data, offset):
    c = offset+1
    for d in data[offset:]:
        print "%d of %d"%(c,len(data))
        features = scrapeYelpFeatures(d["url"])
        d.update(features)
        c += 1
        
def get_feature_set(datalist):
    features = set()
    for f in datalist:
        features |= set(f)
    return features

def fix_features(datalist):
    features = get_feature_set(datalist)
    for d in datalist:
        for f in features:
            if f not in d:
                d[f] = None

def get_random_data(data, n):
    datalist = []
    for d in data:
        datalist += data[d]
    return random.sample(datalist,n)

In [7]:
def retrieve_html(url):
    html = requests.get(url)
    return (html.status_code, html.content)

# Gets restaurant features from yelp html given a url for a restaurants page
# returns a dictionary of the available features on the page (keys) with their 
# corresponding values
def scrapeYelpFeatures(url):
    (yolo, html) = retrieve_html(url)
    
    try:
        imgs =  re.search(r"See all [0-9]+", html).group()
        imgs = [int(s) for s in imgs.split() if s.isdigit()][0]
    except:
        imgs = 0
    root = BeautifulSoup(html, "html.parser")
    s = root.findAll("div", {'class' : 'ywidget'}) #, { "class" : "review review--with-sidebar" })
    #s = root.find("h3", {'name': 'More business info'})
    
    biz_features = {"number_of_images" : imgs}
    for i in s:
        h3_elem = i.find("h3")
        if h3_elem != None:
            if 'More business info' in h3_elem:
                biz_attr = i.find('ul', {'class' : 'ylist'}).findAll('dt', {'class' : 'attribute-key'})
                biz_val = i.find('ul', {'class' : 'ylist'}).findAll('dd')

                for (l,r) in zip(biz_attr, biz_val):
                    #print l.next_element.strip(), '====', r.next_element.strip()
                    biz_features[l.next_element.strip()] = r.next_element.strip()
    return biz_features

In [14]:
info = load_pickle("Anaheim, California features.pkl")

In [ ]:
client = authenticate("config_secret.json")

In [98]:
#datalist = data['Anaheim, California'][:]
#print datalist[999]
#add_feature_info(datalist,268)
#save_pickle("Anaheim, California features.pkl", datalist)

In [104]:
#save_pickle("Anaheim, California features.pkl", datalist)

In [22]:
data = load_pickle("cities_all_restaurants_new.pkl")

In [30]:
datalist = get_random_data(data,50000)[:]

In [ ]:
add_feature_info(datalist,800)

In [34]:
save_pickle("random.pkl", datalist[25000:25000+12500])

In [8]:
datalist = load_pickle("kevin.pkl")

In [9]:
add_feature_info(datalist, 0)

1 of 12500
2 of 12500
3 of 12500
4 of 12500
5 of 12500
6 of 12500
7 of 12500
8 of 12500
9 of 12500
10 of 12500
11 of 12500
12 of 12500
13 of 12500
14 of 12500
15 of 12500
16 of 12500
17 of 12500
18 of 12500
19 of 12500
20 of 12500
21 of 12500
22 of 12500
23 of 12500
24 of 12500
25 of 12500
26 of 12500
27 of 12500
28 of 12500
29 of 12500
30 of 12500
31 of 12500
32 of 12500
33 of 12500
34 of 12500
35 of 12500
36 of 12500
37 of 12500
38 of 12500
39 of 12500
40 of 12500
41 of 12500
42 of 12500
43 of 12500
44 of 12500
45 of 12500
46 of 12500
47 of 12500
48 of 12500
49 of 12500
50 of 12500
51 of 12500
52 of 12500
53 of 12500
54 of 12500
55 of 12500
56 of 12500
57 of 12500
58 of 12500
59 of 12500
60 of 12500
61 of 12500
62 of 12500
63 of 12500
64 of 12500
65 of 12500
66 of 12500
67 of 12500
68 of 12500
69 of 12500
70 of 12500
71 of 12500
72 of 12500
73 of 12500
74 of 12500
75 of 12500
76 of 12500
77 of 12500
78 of 12500
79 of 12500
80 of 12500
81 of 12500
82 of 12500
83 of 12500
84 of 12500
8

KeyboardInterrupt: 

In [ ]:
savedatalist[:968]